In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
lung_cancer_dir = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/LungCancer'

In [5]:
x_rcc_test_file = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/April_30_Finetune_Data/X_finetune_test.csv'

x_rcc_test = pd.read_csv(x_rcc_test_file, index_col=0)

In [6]:


lung_cancer_alignment = pd.read_csv(f'{lung_cancer_dir}/alignment_df.csv', index_col=0)
intensity_matrix = pd.read_csv(f'{lung_cancer_dir}/combined_study.csv', index_col=0)
lung_cancer_metadata = pd.read_csv(f'{lung_cancer_dir}/Lung-Cancer_ST000388_metadata.tsv', index_col=0, sep='\t')

In [7]:
intensity_matrix

,0102_Marios_PD1_Inhib2_HP-E9096038-7.mzML,0525_Marios_PD1_Inhib2_HP-E9811948-6.mzML,0313_Marios_PD1_Inhib2_HP-L1381859-7.mzML,0023_Marios_RCC_HP-000200048-1.mzML,0196_Marios_PD1_Inhib2_HP-E9123617-6.mzML,0211_Marios_RCC_HP-001400059-2.mzML,1002_Marios_PD1_Inhib2_HP-AD195648-7.mzML,0007_Marios_PD1_Inhib2_HP-AF127961-9.mzML,0562_Marios_PD1_Inhib2_HP-AD829004-7.mzML,0600bb_Marios_PD1_Inhib2_HP-PREFB31.mzML,...,LungNodule_HILIC_Pos_47.mzML,LungNodule_HILIC_Pos_117.mzML,LungNodule_HILIC_Pos_80.mzML,LungNodule_HILIC_Pos_18.mzML,LungNodule_HILIC_Pos_122.mzML,LungNodule_HILIC_Pos_41.mzML,LungNodule_HILIC_Pos_109_2.mzML,LungNodule_HILIC_Pos_13.mzML,LungNodule_HILIC_Pos_72.mzML,LungNodule_HILIC_Pos_36.mzML
FT11140,16.148597,17.704074,18.185517,18.157245,17.865694,17.863198,17.861445,17.349323,18.152112,17.807460,...,12.973302,12.641003,12.036566,12.588812,13.493632,13.575455,12.489212,13.053592,12.863241,12.643671
FT9598,18.828817,18.463120,17.335180,15.719428,18.004009,15.719428,15.719428,20.047024,17.151265,18.765442,...,9.966905,10.328328,10.560861,10.020897,10.377888,10.875208,9.632063,9.632063,9.632063,10.452658
FT8972,18.399757,17.612791,21.265537,20.348170,18.285141,19.794531,20.953353,20.375341,21.770559,20.040646,...,10.544021,12.582357,11.106171,10.544021,12.960696,12.287268,13.767520,12.513710,10.544021,10.544021
FT7968,18.936661,19.327889,16.996906,20.187522,16.996906,19.743317,19.018536,16.996906,19.266470,16.996906,...,13.647017,13.493413,12.521993,14.007147,13.116669,13.926644,12.283664,12.883287,13.629282,13.688808
FT393,17.409577,14.143227,17.232691,17.617670,14.143227,16.402037,14.143227,16.169365,17.806739,16.771770,...,11.702850,14.779409,14.463481,13.189535,14.468377,13.260544,14.202823,13.398126,11.626305,11.617634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FT5363,17.663017,16.966477,17.118137,17.311255,17.042691,14.956200,17.385476,17.048912,17.237952,17.187555,...,8.952009,8.952009,8.952009,8.952009,8.952009,8.952009,8.952009,9.609475,10.427030,8.952009
FT711,16.619582,16.961382,17.568094,16.389864,14.181723,16.872531,14.181723,16.495681,16.777980,16.731081,...,10.994721,11.897622,11.924892,11.566985,10.960491,10.416830,12.022609,11.519388,11.355297,10.281094
FT7863,20.578139,20.053739,21.064987,18.064442,19.251521,17.775291,20.529244,20.346149,20.197514,20.129251,...,12.072932,14.764093,11.859964,12.731237,11.377435,12.777008,11.377435,12.598250,12.672613,13.823634
FT6925,14.684214,14.684214,20.405252,19.442389,14.684214,19.654591,20.605273,19.878510,20.636202,19.619114,...,13.300140,13.316662,13.295633,12.442503,13.022085,13.866050,14.174812,12.945745,13.749178,13.483601


In [8]:
lung_cancer_metadata['mzml_file'] = lung_cancer_metadata['file name'].apply(lambda x: x+'.mzML')

In [9]:
lung_cancer_metadata.index = lung_cancer_metadata['mzml_file']

In [10]:
common_feats = [x for x in x_rcc_test.columns if x in intensity_matrix.index]

In [11]:
missing_feats = [x for x in x_rcc_test.columns if x not in intensity_matrix.index]

In [12]:
print(f'number of common features with the pretraining data: {len(common_feats)}')

number of common features with the pretraining data: 1762


In [13]:
lung_cancer_matrix = intensity_matrix.loc[common_feats][lung_cancer_metadata.index]


In [15]:
## Note we are cheating a little bit by standard scaling on the full 
# dataset before we split into train and test

In [14]:
from sklearn.preprocessing import StandardScaler

data_matrix = pd.DataFrame(index=x_rcc_test.columns, columns=lung_cancer_metadata.index)
data_matrix.loc[common_feats] = lung_cancer_matrix.copy()
data_matrix.fillna(data_matrix.mean(),inplace=True)

scaler = StandardScaler()
data_matrix = scaler.fit_transform(data_matrix.T).T
data_matrix = pd.DataFrame(data_matrix, index=x_rcc_test.columns, columns=lung_cancer_metadata.index)

In [16]:
data_matrix

mzml_file,LungNodule_HILIC_Pos_23.mzML,LungNodule_HILIC_Pos_74.mzML,LungNodule_HILIC_Pos_113.mzML,LungNodule_HILIC_Pos_14.mzML,LungNodule_HILIC_Pos_46.mzML,LungNodule_HILIC_Pos_76.mzML,LungNodule_HILIC_Pos_10.mzML,LungNodule_HILIC_Pos_13.mzML,LungNodule_HILIC_Pos_20.mzML,LungNodule_HILIC_Pos_21.mzML,...,LungNodule_HILIC_Pos_83.mzML,LungNodule_HILIC_Pos_111.mzML,LungNodule_HILIC_Pos_33.mzML,LungNodule_HILIC_Pos_56.mzML,LungNodule_HILIC_Pos_63.mzML,LungNodule_HILIC_Pos_100.mzML,LungNodule_HILIC_Pos_12.mzML,LungNodule_HILIC_Pos_125.mzML,LungNodule_HILIC_Pos_80.mzML,LungNodule_HILIC_Pos_85.mzML
FT10031,-1.440839,-1.440839,0.596647,-0.028931,-1.440839,-1.440839,0.811636,-1.440839,0.572805,0.114442,...,1.983813,0.768163,0.172926,-1.440839,-0.858063,0.652656,-1.440839,0.867265,1.552778,0.486284
FT10035,0.790457,-0.852322,1.485158,0.103892,1.692624,-1.289925,-0.401704,-0.124177,0.170975,-0.382203,...,-1.112214,-0.654118,-0.984595,-0.420814,-1.155353,1.700776,0.769766,0.033778,-0.868490,0.000166
FT10037,-2.180661,-1.471448,-0.765021,-0.126822,-0.953374,1.330553,1.133390,0.419890,0.615660,-0.632659,...,1.374036,-1.784317,0.610708,0.481141,0.728461,1.592760,-0.799330,0.534054,1.588637,0.426579
FT10039,-0.764342,-1.255756,0.735165,-0.256252,-1.063872,-0.371080,1.206499,0.417053,-0.445957,0.828698,...,0.972443,-0.171684,1.253524,0.901946,1.060866,-0.480736,0.138871,0.979838,-0.675309,0.442951
FT10041,-0.037295,-0.374419,-0.519070,0.631292,0.487224,-0.530575,0.467870,-1.019618,-1.353084,0.402998,...,0.423590,-1.593199,-0.530270,0.945596,-1.386286,-1.073515,0.203741,-1.187963,-1.236623,0.036453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FT9985,-0.450885,-0.190085,-1.961084,-0.351978,0.125590,1.130735,0.438183,0.080990,0.178407,0.046595,...,0.591218,0.494961,0.303535,-0.535349,1.494717,0.070414,-0.511887,0.182214,1.146807,0.896595
FT9988,-0.764342,-1.255756,0.735165,-0.256252,-1.063872,-0.371080,1.206499,0.417053,-0.445957,0.828698,...,0.972443,-0.171684,1.253524,0.901946,1.060866,-0.480736,0.138871,0.979838,-0.675309,0.442951
FT9989,0.662583,-0.312016,0.370049,0.135372,-0.003272,0.839920,0.444799,0.719838,0.189346,0.196681,...,-0.735944,0.083411,0.353948,1.326917,-0.555551,0.071526,-0.957077,0.172752,1.336216,1.676634
FT9997,1.325510,0.253495,1.297324,0.756033,0.696731,-1.784038,0.556490,0.293124,0.973599,0.213257,...,0.235716,0.544400,-1.155835,-0.713782,-1.829147,0.951813,0.358272,0.812168,-1.539912,0.585204


In [17]:
lung_cancer_metadata['LungCancer BINARY'] = lung_cancer_metadata['Group'].apply(lambda x: 1 if x == 'Cancer' else 0)
y = lung_cancer_metadata['Group']
X = data_matrix.T

In [20]:
# split the data with statified cross validation
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


y = lung_cancer_metadata
X = data_matrix.T
y_target = lung_cancer_metadata['Group']

data_dir = os.path.join(lung_cancer_dir, 'data_v1')

for i, (train_index, test_index) in enumerate(skf.split(X, y_target)):
    fold_dir = os.path.join(data_dir, f'fold_{i}')
    os.makedirs(fold_dir, exist_ok=True)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train.to_csv(os.path.join(fold_dir, f'X_lungcancer_train.csv'))
    X_test.to_csv(os.path.join(fold_dir, f'X_lungcancer_val.csv'))
    y_train.to_csv(os.path.join(fold_dir, f'y_lungcancer_train.csv'))
    y_test.to_csv(os.path.join(fold_dir, f'y_lungcancer_val.csv'))

# save the folds to directory

In [47]:
from sklearn.model_selection import train_test_split
y = lung_cancer_metadata['Group']
X = data_matrix.T
X_train, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

y_train = lung_cancer_metadata.loc[X_train.index]
y_test = lung_cancer_metadata.loc[X_test.index]

In [3]:
data_dir = os.path.join(lung_cancer_dir, 'data_v0')


In [48]:
data_dir = os.path.join(lung_cancer_dir, 'data_v0')
os.makedirs(data_dir, exist_ok=True)

X_train.to_csv(os.path.join(data_dir, 'X_lungcancer_train.csv'))
X_test.to_csv(os.path.join(data_dir, 'X_lungcancer_val.csv'))
y_train.to_csv(os.path.join(data_dir, 'y_lungcancer_train.csv'))
y_test.to_csv(os.path.join(data_dir, 'y_lungcancer_val.csv'))

In [40]:
y

mzml_file
LungNodule_HILIC_Pos_23.mzML     Benign
LungNodule_HILIC_Pos_74.mzML     Benign
LungNodule_HILIC_Pos_113.mzML    Cancer
LungNodule_HILIC_Pos_14.mzML     Cancer
LungNodule_HILIC_Pos_46.mzML     Cancer
                                  ...  
LungNodule_HILIC_Pos_100.mzML    Cancer
LungNodule_HILIC_Pos_12.mzML     Cancer
LungNodule_HILIC_Pos_125.mzML    Cancer
LungNodule_HILIC_Pos_80.mzML     Cancer
LungNodule_HILIC_Pos_85.mzML     Cancer
Name: Group, Length: 94, dtype: object

In [5]:
from main_finetune import finetune_run_wrapper

In [7]:
data_dir = os.path.join(lung_cancer_dir, 'data_v0')
finetune_run_wrapper(data_dir=data_dir,
                     desc_str = 'lungcancer',
                     num_iterations = 10,
                     eval_on_test = False,
                     file_suffix = '_lungcancer')



[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/e/SUR-4041
eval_params_list [{'y_col_name': 'NIVO OS', 'y_head': 'NIVO OS', 'y_cols': ['NIVO OS', 'OS_Event']}, {'y_col_name': 'NIVO OS', 'y_head': 'EVER OS', 'y_cols': ['NIVO OS', 'OS_Event']}, {'y_col_name': 'NIVO OS', 'y_head': 'Both OS', 'y_cols': ['NIVO OS', 'OS_Event']}, {'y_col_name': 'EVER OS', 'y_head': 'NIVO OS', 'y_cols': ['EVER OS', 'OS_Event']}, {'y_col_name': 'EVER OS', 'y_head': 'EVER OS', 'y_cols': ['EVER OS', 'OS_Event']}, {'y_col_name': 'EVER OS', 'y_head': 'Both OS', 'y_cols': ['EVER OS', 'OS_Event']}, {'y_col_name': 'Both OS', 'y_head': 'EVER OS', 'y_cols': ['OS', 'OS_Event']}, {'y_col_name': 'Both OS', 'y_head': 'NIVO OS', 'y_cols': ['OS', 'OS_Event']}, {'y_col_name': 'Both OS', 'y_head': 'Both OS', 'y_cols': ['OS', 'OS_Event']}, {'y_col_name': 'NIVO PFS', 'y_head': 'NIVO PFS', 'y_cols': ['NIVO PFS', 'PFS_Event']}, {'y_col_name': 'NIVO PFS', 'y_head': 'EVER PFS',

('SUR-4041',
 {'trainrun__val__head_LungCancer__on_LungCancer BINARY_AUROC (micro)': [0.7564102564102564,
   0.7435897435897437,
   0.6794871794871795,
   0.7435897435897436,
   0.6025641025641025,
   0.7051282051282052,
   0.576923076923077,
   0.6025641025641026,
   0.5897435897435898,
   0.717948717948718],
  'trainrun__train__head_LungCancer__on_LungCancer BINARY_AUROC (micro)': [0.9916387959866221,
   0.9933110367892977,
   0.9916387959866221,
   0.9899665551839465,
   0.9949832775919734,
   0.9924749163879598,
   0.959866220735786,
   0.9824414715719063,
   0.9849498327759197,
   0.9832775919732442]})